In [1]:
from sympy.codegen.scipy_nodes import powm1
%%capture
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import utils

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=6000)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

Num GPUs Available:  1
2.15.0
1 Physical GPUs, 1 Logical GPUs


In [3]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## Import datasets

### Breast cancer dataset

In [8]:
train_pairs_breast, val_pairs_breast, test_pairs_breast = utils.split_dataset(utils.create_list())

In [5]:
print(f"Number of training pairs: {len(train_pairs_breast)}")
print(f"Number of val pairs: {len(val_pairs_breast)}")
print(f"Number of test_pairs pairs: {len(test_pairs_breast)}")

Number of training pairs: 517
Number of val pairs: 65
Number of test_pairs pairs: 65


In [9]:
train_dataset_breast = utils.create_dataset(train_pairs_breast, augment=True)
val_dataset_breast = utils.create_dataset(val_pairs_breast)
test_dataset_breast = utils.create_dataset(test_pairs_breast)

In [10]:
#batch the dataset and shuffle the training set
batch_size = 8

train_dataset_breast = train_dataset_breast.shuffle(buffer_size=len(train_pairs_breast)).batch(batch_size)
val_dataset_breast = val_dataset_breast.batch(batch_size)
test_dataset_breast = test_dataset_breast.batch(batch_size)

### Skin cancer dataset

In [11]:
train_dataset_skin = utils.create_dataset(utils.create_list_skin("ISIC-2017_Training_Data", "ISIC-2017_Training_Part1_GroundTruth"), augment=True)
val_dataset_skin = utils.create_dataset(utils.create_list_skin("ISIC-2017_Validation_Data", "ISIC-2017_Validation_Part1_GroundTruth"))
test_dataset_skin = utils.create_dataset(utils.create_list_skin("ISIC-2017_Test_v2_Data", "ISIC-2017_Test_v2_Part1_GroundTruth"))

In [12]:
train_dataset_skin = train_dataset_skin.shuffle(buffer_size=2000).batch(batch_size)
val_dataset_skin = val_dataset_skin.batch(batch_size)
test_dataset_skin = test_dataset_skin.batch(batch_size)

### Brain cancer dataset

In [13]:
train_pairs_brain, val_pairs_brain, test_pairs_brain = utils.split_dataset(utils.create_list_brain())
print(f"Number of training pairs: {len(train_pairs_brain)}")
print(f"Number of val pairs: {len(val_pairs_brain)}")
print(f"Number of test_pairs pairs: {len(test_pairs_brain)}")

Number of training pairs: 2451
Number of val pairs: 306
Number of test_pairs pairs: 307


In [14]:
train_dataset_brain = utils.create_dataset(train_pairs_brain, augment=True)
val_dataset_brain = utils.create_dataset(val_pairs_brain)
test_dataset_brain = utils.create_dataset(test_pairs_brain)

In [15]:
train_dataset_brain = train_dataset_brain.shuffle(buffer_size=len(train_pairs_brain)).batch(batch_size)
val_dataset_brain = val_dataset_brain.batch(batch_size)
test_dataset_brain = test_dataset_brain.batch(batch_size)

## Architecture definition

### Basic U-net

In [17]:
def convolutional_layer(input, num_filters, kernel_size=(3, 3), initializer='he_normal'):

    #first convolution
    x = layers.Conv2D(filters=num_filters, kernel_size = kernel_size, padding='same', kernel_initializer=initializer)(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    #second convolution
    x = layers.Conv2D(filters=num_filters, kernel_size=kernel_size, padding='same', kernel_initializer=initializer)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    return x

In [18]:
def basic_unet(input_size, num_filters=16):
    inputs = tf.keras.Input(shape=input_size)

    #encoder block
    first_conv = convolutional_layer(inputs, num_filters)
    first_layer = layers.MaxPool2D(pool_size=(2, 2))(first_conv)
    first_layer = layers.Dropout(0.3)(first_layer)

    second_conv = convolutional_layer(first_layer, num_filters * 2)
    second_layer = layers.MaxPool2D(pool_size=(2, 2))(second_conv)
    second_layer = layers.Dropout(0.3)(second_layer)

    third_conv = convolutional_layer(second_layer, num_filters * 4)
    third_layer = layers.MaxPool2D(pool_size=(2, 2))(third_conv)
    third_layer = layers.Dropout(0.3)(third_layer)

    fourth_conv = convolutional_layer(third_layer, num_filters * 8)
    fourth_layer = layers.MaxPool2D(pool_size=(2, 2))(fourth_conv)
    fourth_layer = layers.Dropout(0.3)(fourth_layer)

    fifth_conv = convolutional_layer(fourth_layer, num_filters * 16)

    #decoder block
    first_deconv = layers.Conv2DTranspose(num_filters * 8, kernel_size=(3, 3), strides=(2, 2), padding='same')(fifth_conv)
    sixth_layer = layers.concatenate([first_deconv, fourth_conv])
    sixth_layer = layers.Dropout(0.3)(sixth_layer)
    sixth_conv = convolutional_layer(sixth_layer, num_filters * 8)

    second_deconv = layers.Conv2DTranspose(num_filters * 4, kernel_size=(3, 3), strides=(2, 2), padding='same')(sixth_conv)
    seventh_layer = layers.concatenate([second_deconv, third_conv])
    seventh_layer = layers.Dropout(0.3)(seventh_layer)
    seventh_conv = convolutional_layer(seventh_layer, num_filters * 4)

    third_deconv = layers.Conv2DTranspose(num_filters * 2, kernel_size=(3, 3), strides=(2, 2), padding='same')(seventh_conv)
    eighth_layer = layers.concatenate([third_deconv, second_conv])
    eighth_layer = layers.Dropout(0.3)(eighth_layer)
    eighth_conv = convolutional_layer(eighth_layer, num_filters * 2)

    fourth_deconv = layers.Conv2DTranspose(num_filters, kernel_size=(3, 3), strides=(2, 2), padding='same')(eighth_conv)
    ninth_layer = layers.concatenate([fourth_deconv, first_conv])
    ninth_layer = layers.Dropout(0.3)(ninth_layer)
    ninth_conv = convolutional_layer(ninth_layer, num_filters)

    #output layer
    output = layers.Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(ninth_conv)

    unet = Model(inputs=inputs, outputs=output)

    return unet

### UC-Transnet

In [23]:
from keras import layers, Model

class ChannelTransformer(layers.Layer):
    def __init__(self, num_heads=4, embed_dim=256, mlp_dim=512, dropout=0.1):
        super(ChannelTransformer, self).__init__()

        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)

        self.mlp = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation="gelu"),
            layers.Dense(embed_dim)
        ])
        self.dropout = layers.Dropout(dropout)

    def call(self, x):
        batch_size = tf.shape(x)[0]
        H, W, C = x.shape[1], x.shape[2], x.shape[3]

        x = tf.reshape(x, (batch_size, -1, C))
        attn_output = self.attention(x, x)
        x = self.norm1(x + attn_output)

        mlp_output = self.mlp(x)
        x = self.norm2(x + mlp_output)

        x = tf.reshape(x, (batch_size, H, W, C))
        return x

# ==============================
# 🔹 Channel-wise Cross Attention
# ==============================
class CCA(layers.Layer):
    def __init__(self, F_g, F_x):
        super(CCA, self).__init__()
        self.mlp_x = tf.keras.Sequential([
            layers.GlobalAveragePooling2D(),
            layers.Dense(F_x, activation='relu')
        ])
        self.mlp_g = tf.keras.Sequential([
            layers.GlobalAveragePooling2D(),
            layers.Dense(F_x, activation='relu')
        ])

    def call(self, g, x):
        channel_att_x = self.mlp_x(x)
        channel_att_g = self.mlp_g(g)

        scale = tf.expand_dims(tf.expand_dims(tf.nn.sigmoid((channel_att_x + channel_att_g) / 2.0), axis=1), axis=1)
        return x * scale

In [28]:
# ==============================
# 🔹 UCTransNet con Unet Pre-addestrato
# ==============================
def UCTransNet(input_shape=(256, 256, 3), pretrained_unet=None):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    x1 = pretrained_unet.get_layer(index=1)(inputs) if pretrained_unet else convolutional_layer(inputs, 32)
    p1 = layers.MaxPooling2D((2, 2))(x1)

    x2 = pretrained_unet.get_layer(index=3)(p1) if pretrained_unet else convolutional_layer(p1, 64)
    p2 = layers.MaxPooling2D((2, 2))(x2)

    x3 = pretrained_unet.get_layer(index=5)(p2) if pretrained_unet else convolutional_layer(p2, 128)
    p3 = layers.MaxPooling2D((2, 2))(x3)

    # Adattiamo p3 a 256 canali prima di passarlo al Transformer
    p3 = layers.Conv2D(256, (1, 1), padding='same')(p3)
    x4 = ChannelTransformer(num_heads=4, embed_dim=256, mlp_dim=512)(p3)

    # Decoder con CCA
    d1 = layers.UpSampling2D((2, 2))(x4)
    d1 = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(d1)

    x3 = layers.Conv2D(128, (1, 1), padding='same')(x3)  # 🔹 Correzione: x3 ora ha 128 canali
    d1 = CCA(128, 128)(d1, x3)
    d1 = layers.Concatenate()([d1, x3])

    d2 = layers.UpSampling2D((2, 2))(d1)
    d2 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(d2)

    x2 = layers.Conv2D(64, (1, 1), padding='same')(x2)  # 🔹 Correzione per x2
    d2 = CCA(64, 64)(d2, x2)
    d2 = layers.Concatenate()([d2, x2])

    d3 = layers.UpSampling2D((2, 2))(d2)
    d3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(d3)

    x1 = layers.Conv2D(32, (1, 1), padding='same')(x1)  # 🔹 Correzione per x1
    d3 = CCA(32, 32)(d3, x1)
    d3 = layers.Concatenate()([d3, x1])

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(d3)

    return Model(inputs, outputs)

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
lr_adapter = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_dice_coefficient", mode = "max", factor=0.1, patience=5, min_lr=0.00001)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_dice_coefficient", mode= "max", patience=10, restore_best_weights=True)

model = UCTransNet()
model.compile(optimizer=optimizer,
                         loss=utils.tversky_loss,
                         metrics=[utils.dice_coefficient, utils.iou, 'accuracy'])

In [36]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_96 (Conv2D)          (None, 256, 256, 16)         448       ['input_6[0][0]']             
                                                                                                  
 batch_normalization_46 (Ba  (None, 256, 256, 16)         64        ['conv2d_96[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_46 (Activation)  (None, 256, 256, 16)         0         ['batch_normalization_46

In [34]:
model.fit(train_dataset_skin, validation_data=val_dataset_skin, epochs=100, callbacks = [lr_adapter, early_stopping])

Epoch 1/100
65/65 [==============================] - 25s 118ms/step - loss: 0.7509 - dice_coefficient: 0.1895 - iou: 0.1059 - accuracy: 0.4937 - val_loss: 0.7866 - val_dice_coefficient: 0.1454 - val_iou: 0.0805 - val_accuracy: 0.6429 - lr: 1.0000e-04
Epoch 2/100
65/65 [==============================] - 9s 107ms/step - loss: 0.7169 - dice_coefficient: 0.2126 - iou: 0.1203 - accuracy: 0.4336 - val_loss: 0.7714 - val_dice_coefficient: 0.1557 - val_iou: 0.0869 - val_accuracy: 0.5448 - lr: 1.0000e-04
Epoch 3/100
65/65 [==============================] - 9s 117ms/step - loss: 0.7056 - dice_coefficient: 0.2189 - iou: 0.1249 - accuracy: 0.4387 - val_loss: 0.7536 - val_dice_coefficient: 0.1667 - val_iou: 0.0940 - val_accuracy: 0.4950 - lr: 1.0000e-04
Epoch 4/100
65/65 [==============================] - 10s 119ms/step - loss: 0.6940 - dice_coefficient: 0.2260 - iou: 0.1292 - accuracy: 0.4328 - val_loss: 0.7394 - val_dice_coefficient: 0.1745 - val_iou: 0.0991 - val_accuracy: 0.4675 - lr: 1.0000e-0

KeyboardInterrupt: 